In [1]:
import cobra
from cobra.flux_analysis import parsimonious
from cobra.flux_analysis.variability import find_essential_genes, find_essential_reactions
from cobra.medium.minimal_medium import minimal_medium

In [2]:
iYL1228 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Klebsiella_2021/data/iYL1228.json')

In [3]:
for x in iYL1228.exchanges: print(x.id)

EX_cd2_e
EX_cbi_e
EX_cbl1_e
EX_cgly_e
EX_cellb_e
EX_chol_e
EX_ca2_e
EX_12ppd__S_e
EX_13ppd_e
EX_14glucan_e
EX_15dap_e
EX_23camp_e
EX_23ccmp_e
EX_23cgmp_e
EX_cit_e
EX_cl_e
EX_23cump_e
EX_23dappa_e
EX_26dap__M_e
EX_2ddglcn_e
EX_cmp_e
EX_co2_e
EX_34dhpac_e
EX_glyc2p_e
EX_3amp_e
EX_3cmp_e
EX_3gmp_e
EX_3hcinnm_e
EX_3hpppn_e
EX_cobalt2_e
EX_colipa_e
EX_crn_e
EX_csn_e
EX_cu_e
EX_cu2_e
EX_cyan_e
EX_cynt_e
EX_3ump_e
EX_4abut_e
EX_glyc3p_e
EX_4hoxpacd_e
EX_cys__D_e
EX_cys__L_e
EX_cytd_e
EX_glyclt_e
EX_dad_2_e
EX_4hphac_e
EX_gmp_e
EX_5dglcn_e
EX_damp_e
EX_dca_e
EX_LalaDgluMdap_e
EX_dcmp_e
EX_gsn_e
EX_dcyt_e
EX_ddca_e
EX_dgmp_e
EX_dgsn_e
EX_LalaDgluMdapDala_e
EX_abt__D_e
EX_ac_e
EX_acald_e
EX_acgal_e
EX_acgal1p_e
EX_dha_e
EX_gthox_e
EX_diact_e
EX_dimp_e
EX_acgam_e
EX_acgam1p_e
EX_din_e
EX_acmana_e
EX_gthrd_e
EX_acmum_e
EX_dms_e
EX_dmso_e
EX_acolipa_e
EX_dopa_e
EX_gtp_e
EX_dtmp_e
EX_acser_e
EX_dump_e
EX_ade_e
EX_gua_e
EX_adn_e
EX_adocbl_e
EX_duri_e
EX_etha_e
EX_ethso3_e
EX_ag_e
EX_etoh_e
EX_h_e
EX_

In [27]:
KPN_loci = []
for x in iYL1228.genes:
    locus = x.id
    locus = locus.split('_')[1]
    KPN_loci.append(locus)
KPN_loci = set(KPN_loci)

KPHS_loci = []
with open('sequence.txt', 'r') as inFile:
    for line in inFile:
        if not line[0] == '>':
            continue
        else:
            locus = line.split()[1]
            locus = locus.split('=')[1]
            locus = locus.rstrip(']')
            locus = locus.split('_')[1]
            KPHS_loci.append(locus)
KPHS_loci = set(KPHS_loci)

In [1]:
# Function to calculate doubling time from objective value
def doublingTime(model):
    with model as m:
        if m.slim_optimize(error_value=0.) < 1e-6:
            print('GENRE has no objective flux')
        else:
            growth = (1. / float(m.slim_optimize())) * 3600.
            print(str(round(growth, 2)) + ' minutes doubling time')


# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)

    #print(str(len(blocked)) + ' total reactions are blocked')
    fraction = (float(len(blocked)) / float(len(model.reactions))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions are blocked')
    
    return blocked


# Identify potentially gapfilled reactions, checks against pFBA solution
def missingGPR(model):
    
    noGene = []
    exclude = []
    for rxn in model.reactions:
        if len(list(rxn.genes)) == 0:
            if rxn.annotation['sbo'] != 'SBO:0000629':
                if rxn in model.boundary:
                    exclude.append(rxn.id)
                    continue
                else:
                    noGene.append(rxn.id)
    
    solution = parsimonious.pfba(model)
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-5])
    active_rxns = active_rxns.difference(set(exclude))
    noGene_active = set(noGene).intersection(active_rxns)

    fraction = float(len(model.reactions)) - float(len(exclude))
    fraction = (float(len(noGene)) / fraction) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions without GPRs')
    
    fraction = (float(len(noGene_active)) / float(len(active_rxns))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% of reactions used in pFBA solution have no GPR')
    
    return noGene_active


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(model, cytosol='cytosol'):

    free = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.boundary: m.reactions.get_by_id(rxn.id).lower_bound = 0.
    
        # Create demand for each reaction and optimize individually
        reset_rxn = m.reactions[0].id
        for cpd in m.metabolites: 
            if cpd.compartment == cytosol:
                demand = cobra.Reaction('demand')
                demand.bounds = (0., 1000.)
                demand.add_metabolites({cpd: -1.0})
                m.add_reactions([demand])
                m.objective = demand
                obj_val = m.slim_optimize()
                if obj_val > 1e-8: free.append(cpd.id)
                m.objective = reset_rxn
                m.remove_reactions([demand])
    
    fraction = (float(len(free)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(model):
    
    with model as m:

        elements = set()
        for cpd in m.metabolites:
            try:
                elements |= set(cpd.elements.keys())
            except:
                pass
        
        massImbal = []
        failed = 0
        if len(elements) == 0:
            print('No elemental data associated with metabolites!')
            failed = 1
        else:
            for rxn in m.reactions:
                if rxn.annotation['sbo'] == 'SBO:0000629': 
                    continue
                elif rxn in m.boundary:
                    continue

                try:
                    test = rxn.check_mass_balance()
                except ValueError:
                    continue

                if len(list(test)) > 0:
                    if len(set(test.keys()).intersection(elements)) > 0: massImbal.append(rxn.id)
                        
    if failed != 1:
        fraction = (float(len(massImbal)) / float(len(model.reactions))) * 100.
        fraction = round(fraction, 2)
        print(str(fraction) + '% reactions are mass imbalanced')
        
    return massImbal


def basicCheck(model):
    
    # Determination
    if len(model.reactions) < len(model.metabolites): 
        print('GENRE is overdetermined')
    elif len(model.reactions) > len(model.metabolites):
        print('GENRE is underdetermined')
    else:
        pass
    
    # Compartments
    print('GENRE has ' + str(len(model.compartments.keys())) + ' compartment(s)')
    
    # Genes
    if len(model.genes) == 0: 
        print('GENRE has no gene data')
    else:
        print('GENRE has ' + str(len(model.genes)) + ' genes')
          
    # Growth
    doublingTime(model)



In [5]:
# Open all exchange bounds
for x in iYL1228.exchanges: x.bounds = (-1000., 1000.)

In [10]:
basicCheck(iYL1228)
draft_noGPRblocked = blockedReactions(iYL1228)
draft_free = checkFreeMass(iYL1228)
draft_massImbal = checkBalance(iYL1228)
draft_nogene = missingGPR(iYL1228)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 1229 genes
53.94 minutes doubling time
22.37% reactions are blocked
0.0% metabolites are generated for free
No elemental data associated with metabolites!
3.86% reactions without GPRs
2.63% of reactions used in pFBA solution have no GPR


In [1]:
from riptide import *

In [9]:
iYL1228 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Klebsiella_2021/data/iYL1228.json')

In [3]:
HBF_medium_open = {'EX_glc_e':-1000, 'EX_ala__L_e':-1000, 'EX_arg__L_e':-1000, 'EX_asp__L_e':-1000, 'EX_cys__L_e':-1000, 
             'EX_glu__L_e':-1000, 'EX_gly_e':-1000, 'EX_his__L_e':-1000, 'EX_ile__L_e':-1000, 'EX_leu__L_e':-1000, 
             'EX_lys__L_e':-1000, 'EX_met__L_e':-1000, 'EX_pro__L_e':-1000, 'EX_thr__L_e':-1000, 'EX_tyr__L_e':-1000, 
             'EX_phe__L_e':-1000, 'EX_ser__L_e':-1000, 'EX_trp__L_e':-1000, 'EX_val__L_e':-1000, 'EX_pnto_R_e':-1000, 
             'EX_nac_e':-1000, 'EX_na1_e':-1000, 'EX_cl_e':-1000, 'EX_so4_e':-1000, 'EX_k_e':-1000, 
             'EX_pi_e':-1000, 'EX_ca2_e':-1000, 'EX_mg2_e':-1000, 'EX_zn2_e':-1000, 'EX_aso3_e':-1000, 
             'EX_cd2_e':-1000, 'EX_hg2_e':-1000, 'EX_h_e':-1000, 'EX_h2o_e':-1000, 'EX_o2_e':-1000, 
             'EX_ins_e':-5, 'EX_hxan_e':-5, 'EX_dcyt_e':-5, 'EX_thymd_e':-5, 'EX_ura_e':-5, 'EX_uri_e':-1000, 
             'EX_dad_2_e':-5, 'EX_adn_e':-5, 'EX_co2_e':-1000, 'EX_cobalt2_e':-1000, 'EX_cu2_e':-1000, 
             'EX_fe2_e':-1000, 'EX_fe3_e':-1000, 'EX_mn2_e':-1000, 'EX_mobd_e':-1000, 'EX_tungs_e':-1000, 
             'EX_cbl1_e':-1000, 'EX_fru_e':-1000, 'EX_gal_e':-1000, 'EX_ni2_e':-1000, 'EX_sel_e':-1000, 
             'EX_slnt_e':-1000} 

HBF_medium = {'EX_glc_e':-5, 'EX_ala__L_e':-5, 'EX_arg__L_e':-5, 'EX_asp__L_e':-5, 'EX_cys__L_e':-5, 
             'EX_glu__L_e':-5, 'EX_gly_e':-5, 'EX_his__L_e':-5, 'EX_ile__L_e':-5, 'EX_leu__L_e':-5, 
             'EX_lys__L_e':-5, 'EX_met__L_e':-5, 'EX_pro__L_e':-5, 'EX_thr__L_e':-5, 'EX_tyr__L_e':-5, 
             'EX_phe__L_e':-5, 'EX_ser__L_e':-5, 'EX_trp__L_e':-5, 'EX_val__L_e':-5, 'EX_pnto_R_e':-5, 
             'EX_nac_e':-5, 'EX_na1_e':-1000, 'EX_cl_e':-1000, 'EX_so4_e':-1000, 'EX_k_e':-1000, 
             'EX_pi_e':-1000, 'EX_ca2_e':-1000, 'EX_mg2_e':-1000, 'EX_zn2_e':-1000, 'EX_aso3_e':-1000, 
             'EX_cd2_e':-1000, 'EX_hg2_e':-1000, 'EX_h_e':-100, 'EX_h2o_e':-100, 'EX_o2_e':-18.5, 
             'EX_ins_e':-5, 'EX_hxan_e':-5, 'EX_dcyt_e':-5, 'EX_thymd_e':-5, 'EX_ura_e':-5, 'EX_uri_e':-5, 
             'EX_dad_2_e':-5, 'EX_adn_e':-5, 'EX_co2_e':-1000, 'EX_cobalt2_e':-1000, 'EX_cu2_e':-1000, 
             'EX_fe2_e':-1000, 'EX_fe3_e':-1000, 'EX_mn2_e':-1000, 'EX_mobd_e':-1000, 'EX_tungs_e':-1000, 
             'EX_cbl1_e':-0.01, 'EX_fru_e':-5, 'EX_gal_e':-5, 'EX_ni2_e':-1000, 'EX_sel_e':-1000, 
             'EX_slnt_e':-1000} 

In [4]:
# Set media conditions
for x in iYL1228.exchanges:
    try:
        x.bounds = (float(HBF_medium[x.id]), 1000.)
    except:
        x.bounds = (0., 1000.)

In [5]:
clinical = riptide.read_transcription_file(file='/home/mjenior/Desktop/active_projects/klebsiella/data/transcript_mapping/clinical_replicates.tsv')

In [6]:
iYL1228_clinical = riptide.maxfit_contextualize(model=iYL1228, transcriptome=clinical)


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.115 ; p = 0.0208
Fraction = 0.4 | Rho = 0.107 ; p = 0.031
Fraction = 0.45 | Rho = 0.118 ; p = 0.0172
Fraction = 0.5 | Rho = 0.0671 ; p = 0.1759
Fraction = 0.55 | Rho = 0.0639 ; p = 0.1994
Fraction = 0.6 | Rho = 0.0794 ; p = 0.1112
Fraction = 0.65 | Rho = 0.061 ; p = 0.2238
Fraction = 0.7 | Rho = 0.0769 ; p = 0.1261
Fraction = 0.75 | Rho = 0.1186 ; p = 0.0197
Fraction = 0.8 | Rho = 0.1106 ; p = 0.0307
Fraction = 0.85 | Rho = 0.1212 ; p = 0.0175
Fraction = 0.9 | Rho = 0.1183 ; p = 0.0193
Testing local objective fractions to 0.85...
Fraction = 0.825 | Rho = 0.1178 ; p = 0.0215
Fraction = 0.875 | Rho = 0.1081 ; p = 0.0343

Context-specific metabolism fit with 0.85 of optimal objective flux

Reactions pruned to 384 from 2262 (83.02% change)
Metabolites pruned to 354 from 1658 (78.65% change)
Flux through the objective DECREASED to ~2.5

In [7]:
riptide.save_output(iYL1228_clinical, path='/home/mjenior/Desktop/repos/Klebsiella_2021/data/clinical_maxfit_reps')

In [5]:
laboratory = read_transcription_file(file='/home/mjenior/Desktop/active_projects/klebsiella/data/transcript_mapping/laboratory_replicates.tsv')

In [6]:
iYL1228_laboratory = riptide.maxfit_contextualize(model=iYL1228, transcriptome=laboratory)


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1911 ; p = 0.0001
Fraction = 0.4 | Rho = 0.2091 ; p = 0.0
Fraction = 0.45 | Rho = 0.1659 ; p = 0.0009
Fraction = 0.5 | Rho = 0.1401 ; p = 0.0054
Fraction = 0.55 | Rho = 0.1849 ; p = 0.0002
Fraction = 0.6 | Rho = 0.2058 ; p = 0.0001
Fraction = 0.65 | Rho = 0.163 ; p = 0.0015
Fraction = 0.7 | Rho = 0.1504 ; p = 0.0033
Fraction = 0.75 | Rho = 0.1726 ; p = 0.0007
Fraction = 0.8 | Rho = 0.1697 ; p = 0.0008
Fraction = 0.85 | Rho = 0.2088 ; p = 0.0
Fraction = 0.9 | Rho = 0.1891 ; p = 0.0001
Testing local objective fractions to 0.4...
Fraction = 0.375 | Rho = 0.214 ; p = 0.0
Fraction = 0.425 | Rho = 0.1561 ; p = 0.0019

Context-specific metabolism fit with 0.375 of optimal objective flux

Reactions pruned to 393 from 2262 (82.63% change)
Metabolites pruned to 350 from 1658 (78.89% change)
Flux through the objective DECREASED to ~2.26 from

In [7]:
# Save output
riptide.save_output(iYL1228_laboratory, path='/home/mjenior/Desktop/repos/Klebsiella_2021/data/laboratory_maxfit_reps')

In [2]:
iYL1228_clinical = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Klebsiella_2021/data/clinical_maxfit_reps/model.sbml')
iYL1228_laboratory = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Klebsiella_2021/data/laboratory_maxfit_reps/model.sbml')

In [4]:
iYL1228_clinical

Name,iYL1228_riptide
Memory address,0x07fac256c2f10
Number of metabolites,354
Number of reactions,384
Number of groups,0
Objective expression,1.0*BIOMASS_ - 1.0*BIOMASS__reverse_091e5
Compartments,"cytosol, periplasm, extracellular space"


In [5]:
iYL1228_laboratory

Name,iYL1228_riptide
Memory address,0x07fac256c2d50
Number of metabolites,350
Number of reactions,393
Number of groups,0
Objective expression,1.0*BIOMASS_ - 1.0*BIOMASS__reverse_091e5
Compartments,"cytosol, periplasm, extracellular space"


In [6]:
clinical_rxns = set([x.id for x in iYL1228_clinical.reactions])
laboratory_rxns = set([y.id for y in iYL1228_laboratory.reactions])

clinical_only_rxns = clinical_rxns.difference(laboratory_rxns)
print('Clinical only:',len(clinical_only_rxns))
laboratory_only_rxns = laboratory_rxns.difference(clinical_rxns)
print('Laboratory only:',len(laboratory_only_rxns))

Clinical only: 39
Laboratory only: 48


In [10]:
for x in clinical_only_rxns: print(x, iYL1228.reactions.get_by_id(x).name)

PGP181abcpp Phosphatidylglycerophosphate transport via ABC system (n-C18:1, periplasm)
VPAMTr Valine-pyruvate aminotransferase
F6PA Fructose 6-phosphate aldolase
URIDK2r Uridylate kinase (dUMP)
KARA1 Ketol-acid reductoisomerase (2,3-dihydroxy-3-methylbutanoate)
SERD_L L-serine deaminase
PGPP160pp Phosphatidylglycerol phosphate phosphatase (periplasm, n-C16:0)
EX_h2_e H2 exchange
FRD3 Fumarate reductase
RNDR4 Ribonucleoside-diphosphate reductase (UDP)
DADK Deoxyadenylate kinase
HXPRT Hypoxanthine phosphoribosyltransferase (Hypoxanthine)
G5SD Glutamate-5-semialdehyde dehydrogenase
VALTA Valine transaminase
GLU5K Glutamate 5-kinase
P5CR Pyrroline-5-carboxylate reductase
EX_ala__D_e D-Alanine exchange
ADSL1r Adenylsuccinate lyase
H2tpp Hydrogen transport diffusion (periplasm)
DALAtex D-Alanine transport via diffusion (extracellular to periplasm)
PGPP161pp Phosphatidylglycerol phosphate phosphatase (periplasm, n-C16:1)
GLYCL Glycine Cleavage System
NTD6 5'-nucleotidase (dAMP)
PGP161abcpp Ph

In [11]:
for x in laboratory_only_rxns: print(x, iYL1228.reactions.get_by_id(x).name)

MTHFD Methylenetetrahydrofolate dehydrogenase (NADP)
EAR181y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C18:1)
ILEabcpp L-isoleucine transport via ABC system (periplasm)
FORtppi Formate transport via diffusion (cytoplasm to periplasm)
ALAtex L-alanine transport via diffusion (extracellular to periplasm)
EX_for_e Formate exchange
IZPN Imidazolonepropionase
MTHFC Methenyltetrahydrofolate cyclohydrolase
EAR60y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C6:0)
EAR161y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C16:1)
FTHFD Formyltetrahydrofolate deformylase
FGLU Formimidoylglutamase
PGCD Phosphoglycerate dehydrogenase
ICL Isocitrate lyase
HYD1pp Hydrogenase (ubiquinone-8: 2 protons) (periplasm)
FORAMD FORAMD
AKGDH 2-Oxogluterate dehydrogenase
EAR140y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C14:0)
EAR100y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C10:0)
PSERT Phosphoserine transaminase
EAR80y Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C8:0)
GLYCL

In [14]:
from cobra.flux_analysis.variability import find_essential_genes, find_essential_reactions

In [15]:
minGrowth = iYL1228.slim_optimize() * 0.01
core_essential_genes = find_essential_genes(iYL1228, threshold=minGrowth)
core_essential_genes = set([x.id for x in core_essential_genes])
print(str(len(core_essential_genes)) + ' core essential genes found')

119 core essential genes found


In [16]:
minGrowth = iYL1228_clinical.slim_optimize() * 0.01
clinical_essential_genes = find_essential_genes(iYL1228_clinical, threshold=minGrowth)
clinical_essential_genes = set([x.id for x in clinical_essential_genes]).difference(core_essential_genes)
print(str(len(clinical_essential_genes)) + ' clinical context-specific essential genes found')

41 clinical context-specific essential genes found


In [17]:
minGrowth = iYL1228_laboratory.slim_optimize() * 0.01
laboratory_essential_genes = find_essential_genes(iYL1228_laboratory, threshold=minGrowth)
laboratory_essential_genes = set([x.id for x in laboratory_essential_genes]).difference(core_essential_genes)
print(str(len(laboratory_essential_genes)) + ' laboratory context-specific essential genes found')

16 laboratory context-specific essential genes found


In [18]:
clinical_only_essential_genes = clinical_essential_genes.difference(laboratory_essential_genes)
print(str(len(clinical_only_essential_genes)) + ' clinical-only context-specific essential genes found')
laboratory_only_essential_genes = laboratory_essential_genes.difference(clinical_essential_genes)
print(str(len(laboratory_only_essential_genes)) + ' laboratory-only context-specific essential genes found')

28 clinical-only context-specific essential genes found
3 laboratory-only context-specific essential genes found


In [24]:
for x in clinical_only_essential_genes:
    for y in iYL1228.genes.get_by_id(x).reactions:
        print(y.id, y.name)

NADHNQR NADH: quinone oxidoreductas, NQR-type
ATPS4rpp ATP synthase (four protons for one ATP) (periplasm)
GLCOAS Glutaryl-CoA synthetase
SUCOAS Succinyl-CoA synthetase (ADP-forming)
CYTBO3_4pp Cytochrome oxidase bo3 (ubiquinol-8: 4 protons) (periplasm)
DRPA Deoxyribose-phosphate aldolase
NADHNQR NADH: quinone oxidoreductas, NQR-type
ATPS4rpp ATP synthase (four protons for one ATP) (periplasm)
ATPS4rpp ATP synthase (four protons for one ATP) (periplasm)
PGK Phosphoglycerate kinase
CYTDt2pp Cytidine transport in via proton symport (periplasm)
DURIt2pp Deoxyuridine transport in via proton symport (periplasm)
THMDt2pp Thymidine transport in via proton symport (periplasm)
DCYTt2pp Deoxycytidine transport in via proton symport (periplasm)
INSt2pp Inosine transport in via proton symport (periplasm)
DGSNt2pp Deoxyguanosine transport in via proton symport (periplasm)
URIt2pp Uridine transport in via proton symport (periplasm)
DINSt2pp Deoxyinosine transport in via proton symport (periplasm)
DA

In [25]:
for x in laboratory_only_essential_genes: 
    for y in iYL1228.genes.get_by_id(x).reactions:
        print(y.id, y.name)

DSERt2pp D-serine transport in via proton symport (periplasm)
DALAt2pp D-alanine transport in via proton symport (periplasm)
GLYt2pp Glycine transport in via proton symport (periplasm)
ALAt2pp L-alanine transport in via proton symport (periplasm)
BALAt2pp Beta-alanine transport in via proton symport (periplasm)
DURIK1 Deoxyuridine kinase (ATP:Deoxyuridine)
TMDK1 Thymidine kinase (ATP:thymidine)
INSK Insosine kinase
GSNK Guanosine kinase


In [26]:
for x in core_essential_genes:
    for y in iYL1228.genes.get_by_id(x).reactions:
        print(y.id, y.name)

4HTHRS 4-Hydroxy-L-threonine synthase
THRS Threonine synthase
UAGDP UDP-N-acetylglucosamine diphosphorylase
G1PACT Glucosamine-1-phosphate N-acetyltransferase
DASYN120 CDP-diacylglycerol synthetase (n-C12:0)
DASYN141 CDP-diacylglycerol synthetase (n-C14:1)
DASYN160 CDP-diacylglycerol synthetase (n-C16:0)
DASYN140 CDP-diacylglycerol synthetase (n-C14:0)
DASYN181 CDP-diacylglycerol synthetase (n-C18:1)
DASYN180 CDP-diacylglycerol synthetase (n-C18:0)
DASYN161 CDP-diacylglycerol synthetase (n-C16:1)
UAMAGS UDP-N-acetylmuramoyl-L-alanyl-D-glutamate synthetase
DHDPS Dihydrodipicolinate synthase
ATPPRT ATP phosphoribosyltransferase
ADSL1r Adenylsuccinate lyase
ADSL2r Adenylosuccinate lyase
DHQS 3-dehydroquinate synthase
PAPSR2 Phosphoadenylyl-sulfate reductase (glutaredoxin)
PAPSR Phosphoadenylyl-sulfate reductase (thioredoxin)
GLUPRT Glutamine phosphoribosyldiphosphate amidotransferase
IPPMIa 3-isopropylmalate dehydratase
IPPMIb 2-isopropylmalate hydratase
KARA2 Ketol-acid reductoisomerase 